Scenario
You’ve been hired by a startup incubator to build an AI-powered assistant that helps aspiring entrepreneurs brainstorm business ideas, evaluate their potential, and summarize key insights.

_Challenge_
Create an AI Business Advisor that:
Accepts an industry as input.
Generates a business idea.
Analyzes the strengths and weaknesses.
Formats the results as a final report.

Use LangChain LCEL to chain prompts, LLMs, and output parsers.

## 0. Import the necessary libs

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field


## 1. Instantiate Chat Model with your API Key

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,  # no api_key arg needed if env var is set, Env var name must be exact: OPENAI_API_KEY (uppercase). If you set openai_api_key anywhere, rename it.
)

## 2. Your first Chain
In the end of each chain, you should parse the output and save the logs

In [4]:
logs = []
parser = StrOutputParser()
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation
Craft a prompt to generate a business idea for the given industry.

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [5]:
idea_prompt = PromptTemplate(
    template=(
        "You are a creative business advisor. "
        "Generate one innovative business idea in the industry: "
        "{industry}. "
        "Provide a brief description of the idea."
    )
)

In [6]:
idea_chain = (
    idea_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [7]:
idea_result = idea_chain.invoke("agro")

In [8]:
idea_result["output"]

'**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments and small-scale farmers. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops year-round, regardless of external weather conditions. Equipped with IoT sensors, the pods monitor and optimize factors such as nutrient levels, humidity, temperature, and light exposure, ensuring maximum yield with minimal resource use.\n\nThe SVFPs can be installed in backyards, rooftops, or community spaces, making fresh produce accessible to urban dwellers. Users can control their pods via a mobile app, which provides real-time data, growth tracking, and automated care schedules. Additionally, the app includes a community feature where users can share tips, recipes, and even trade produce with neighbors.\n\nTo enhance sustainability, the pods are designed to use renewable energy sources, s

In [9]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments and small-scale farmers. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops year-round, regardless of external weather conditions. Equipped with IoT sensors, the pods monitor and optimize factors such as nutrient levels, humidity, temperature, and light exposure, ensuring maximum yield with minimal resource use.\n\nThe SVFPs can be installed in backyards, rooftops, or community spaces, making fresh produce accessible to urban dwellers. Users can control their pods via a mobile app, which provides real-time data, growth tracking, and automated care schedules. Additionally, the app includes a community feature where users can share tips, recipes, and even trade produce with neighbors.\n\nTo enhance sustainability, the pods are designed to use renewabl

## 4. Idea Analysis
Craft a prompt to analyze the generated idea's strengths and weaknesses

In [10]:
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the following business idea: "
        "Idea: {idea} "
        "Identify 3 key strengths and 3 potential weaknesses of the idea."
    )
)

In [11]:
analysis_chain = (
    analysis_prompt 
    | llm 
    | parse_and_log_output_chain
)

In [12]:
analysis_result = analysis_chain.invoke(idea_result["output"])

In [13]:
analysis_result["output"]

"### Key Strengths\n\n1. **Sustainability and Resource Efficiency**: The Smart Vertical Farming Pods (SVFP) utilize advanced hydroponic and aeroponic systems, which are known for their efficient use of water and nutrients compared to traditional farming. The incorporation of renewable energy sources and rainwater collection further enhances sustainability, appealing to environmentally conscious consumers and aligning with global trends toward sustainable living.\n\n2. **Urban Food Security and Accessibility**: By enabling urban dwellers to grow their own food in limited spaces, SVFP addresses the growing concern of food security in cities. The modular design allows for installation in various urban settings, making fresh produce more accessible and reducing the carbon footprint associated with transporting food from rural farms to urban markets.\n\n3. **Community Engagement and Education**: The mobile app's community features encourage interaction among users, fostering a sense of comm

In [14]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments and small-scale farmers. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops year-round, regardless of external weather conditions. Equipped with IoT sensors, the pods monitor and optimize factors such as nutrient levels, humidity, temperature, and light exposure, ensuring maximum yield with minimal resource use.\n\nThe SVFPs can be installed in backyards, rooftops, or community spaces, making fresh produce accessible to urban dwellers. Users can control their pods via a mobile app, which provides real-time data, growth tracking, and automated care schedules. Additionally, the app includes a community feature where users can share tips, recipes, and even trade produce with neighbors.\n\nTo enhance sustainability, the pods are designed to use renewabl

## 5. Report Generation
Craft a prompt to structure the information into a business report.

In [15]:
report_prompt = PromptTemplate(
    template=(
        "Here is a business analysis: "
        "Strengths & Weaknesses: {output} "
        "Generate a structured business report." 
    )
)

In [16]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [17]:
report_chain = (
    report_prompt | llm.with_structured_output(schema=AnalysisReport, method="function_calling")
)

In [18]:
report_result = report_chain.invoke(analysis_result["output"])

In [19]:
report_result

AnalysisReport(strengths=['Sustainability and Resource Efficiency: Utilizes advanced hydroponic and aeroponic systems for efficient water and nutrient use, appealing to environmentally conscious consumers.', 'Urban Food Security and Accessibility: Enables urban dwellers to grow their own food, addressing food security concerns and reducing carbon footprint associated with food transport.', 'Community Engagement and Education: Mobile app fosters user interaction and knowledge sharing, enhancing user experience and promoting brand loyalty.'], weaknesses=['High Initial Investment and Maintenance Costs: Upfront costs may deter potential users, especially in lower-income areas, along with ongoing maintenance costs.', 'Technical Complexity and User Adoption: IoT features may intimidate less tech-savvy users, necessitating user-friendly design and support.', 'Market Competition and Scalability: Increasing competition in vertical farming requires differentiation and poses challenges in scaling

## 6. End to End Chain

In [20]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [21]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [22]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [24]:
e2e_result

AnalysisReport(strengths=['Sustainability and Local Food Production', 'Technological Integration', 'Community Engagement and Education'], weaknesses=['High Initial Investment', 'Maintenance and Technical Issues', 'Market Competition and Adoption'])

In [25]:
e2e_result.strengths

['Sustainability and Local Food Production',
 'Technological Integration',
 'Community Engagement and Education']

In [27]:
e2e_result.weaknesses

['High Initial Investment',
 'Maintenance and Technical Issues',
 'Market Competition and Adoption']

In [28]:
logs

[AIMessage(content='**Business Idea: Smart Vertical Farming Pods (SVFP)**\n\n**Description:**\nSmart Vertical Farming Pods (SVFP) are modular, self-sustaining farming units designed for urban environments and small-scale farmers. Each pod utilizes advanced hydroponic and aeroponic systems to grow a variety of crops year-round, regardless of external weather conditions. Equipped with IoT sensors, the pods monitor and optimize factors such as nutrient levels, humidity, temperature, and light exposure, ensuring maximum yield with minimal resource use.\n\nThe SVFPs can be installed in backyards, rooftops, or community spaces, making fresh produce accessible to urban dwellers. Users can control their pods via a mobile app, which provides real-time data, growth tracking, and automated care schedules. Additionally, the app includes a community feature where users can share tips, recipes, and even trade produce with neighbors.\n\nTo enhance sustainability, the pods are designed to use renewabl

## 7. Experiment
Now that you understood how it works, experiment with new things.

Improve memory
Explore the Runnables
Add More Complexity